# **Fully Connected Neural Network: A `CUDA` and `C++` Implementation**

## **Prepare workspace**

In [76]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/PP/prj

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/PP/prj


## **Extract `.gz` data (if needed)**

In [ ]:
# Extract data from `.gz`
# Only need to run once!
!pip install patool
import patoolib
patoolib.extract_archive("mnist/t10k-images-idx3-ubyte.gz", outdir="mnist")
patoolib.extract_archive("mnist/t10k-labels-idx1-ubyte.gz", outdir="mnist")
patoolib.extract_archive("mnist/train-images-idx3-ubyte.gz", outdir="mnist")
patoolib.extract_archive("mnist/train-labels-idx1-ubyte.gz", outdir="mnist")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 2.0 MB/s eta 0:00:00


INFO patool: Extracting mnist/t10k-images-idx3-ubyte.gz ...
INFO:patool:Extracting mnist/t10k-images-idx3-ubyte.gz ...
INFO patool: running /usr/bin/7z e -omnist -- mnist/t10k-images-idx3-ubyte.gz
INFO:patool:running /usr/bin/7z e -omnist -- mnist/t10k-images-idx3-ubyte.gz
INFO patool: ... mnist/t10k-images-idx3-ubyte.gz extracted to `mnist'.
INFO:patool:... mnist/t10k-images-idx3-ubyte.gz extracted to `mnist'.
INFO patool: Extracting mnist/t10k-labels-idx1-ubyte.gz ...
INFO:patool:Extracting mnist/t10k-labels-idx1-ubyte.gz ...
INFO patool: running /usr/bin/7z e -omnist -- mnist/t10k-labels-idx1-ubyte.gz
INFO:patool:running /usr/bin/7z e -omnist -- mnist/t10k-labels-idx1-ubyte.gz
INFO patool: ... mnist/t10k-labels-idx1-ubyte.gz extracted to `mnist'.
INFO:patool:... mnist/t10k-labels-idx1-ubyte.gz extracted to `mnist'.
INFO patool: Extracting mnist/train-images-idx3-ubyte.gz ...
INFO:patool:Extracting mnist/train-images-idx3-ubyte.gz ...
INFO patool: running /usr/bin/7z e -omnist -- mni

'mnist'

## **Edit `Makefile`**

In [ ]:
%%writefile Makefile

# Compilers
CXX := g++
CXX_FLAGS := -std=c++17 -ggdb
NVCC := nvcc

# Folders
BIN := bin
SRC := src
INCLUDE := include

EXECUTABLE := nn_main

all: $(BIN)/$(EXECUTABLE)

run: clean all
	clear
	./$(BIN)/$(EXECUTABLE)

$(BIN)/$(EXECUTABLE): $(SRC)/*.cu $(SRC)/*.cpp
	$(NVCC) -I $(INCLUDE) $^ -o $@

clean:
	-rm $(BIN)/*

Overwriting Makefile


## **Compile and run**

In [84]:
# Compile
!make

nvcc -I include src/main.cu src/nn.cu src/utils_device.cu src/data.cpp src/utils_host.cpp -o bin/nn_main


In [85]:
# Run the program
# ./main <#-neurons> <#-epochs> <learning-rate>
!./bin/nn_main 20 1 0.5

-- # neurons: 20
-- # epochs: 1
-- learning rate: 0.5
Train Images: 60000 with size 784
Train Labels: 60000 labels loaded
Test Images: 10000 with size 784
Test Labels: 10000 labels loaded

- layer 0 forward time: 3077.616943 ms
- layer 1 forward time: 81.679420 ms
- layer 2 forward time: 51.484959 ms
FORWARD TIME CPU: 3348.668457 ms

- layer 0 forward time: 55.036194 ms
- layer 1 forward time: 5.806560 ms
- layer 2 forward time: 2.578720 ms
FORWARD TIME GPU: 342.502838 ms

- layer 0 forward time: 51.611423 ms
- layer 1 forward time: 5.431424 ms
- layer 2 forward time: 3.412992 ms
FORWARD TIME GPU (OPTIMIZED): 340.794739 ms

-- Mean error CPU - GPU: 0.000198097
-- Mean error CPU - GPU (optimized): 0.000172152

CPU Train start...
-- number of epochs: 1
- layer 0 forward time: 4381.930176 ms
- layer 1 forward time: 138.991043 ms
- layer 2 forward time: 86.502403 ms
>>> Epoch 1 CEE loss: 6.76222
TRAIN TIME: 9548.084961 ms


GPU Train start...
-- number of epochs: 1
- layer 0 forward time: 